In [ ]:
from openai import OpenAI
# from anthropic import Anthropic
import time
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import date
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')

💚 Today is 2025-03-06


In [2]:
# OpenAI
load_dotenv() 
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)

In [3]:
# Data with doi
# data = pd.read_csv(f"{path}/data_abstracts/merged_2025-02-24_gpt_filtered.csv")
data = pd.read_csv(f"{path}/data_abstracts/filter5_2025-02-26_domain.csv")
print(data.shape)

(7994, 15)


# GPT: Study type categorization

In [9]:
gpt_model = "gpt-4o-2024-08-06"
# "gpt-4o-mini-2024-07-18" # Tried on 2025-02-28 for filter 5
# "gpt-4o-mini-2024-07-18" # Tried on 2025-02-20/21/24 for df1-3 merged data
# "gpt-4o-2024-08-06" # Tried on 2025-02-24 for df1-3 merged data and df4

In [5]:
def gpt_extract_abstract(url, gpt_model, abstract):
    response = openai_client.chat.completions.create(
        model= gpt_model,
        messages=[
            {
                "role": "assistant", 
                "content": f"""You're an helpful assistant who can recognize whether this paper {url} studies intergenerational mobility in wealth, income or earnings empirically with real-life datasets or theoretically by delving into the models to measure the mobility. This is the abstract of the paper: {abstract}"""
            },
            {
                "role": "user", 
                "content": 
                        """
                        If the study is empirical return a term: "empirical".
                        If the study is theoretical return a term: "theoretical".
                        If the study is both empirical and theoretical return a term: "both".
                        If the study is not at all about intergenerational wealth mobility return a term: "false".
                        ```plaintext
                        """
            }
        ],
        max_tokens=100,
        temperature=0.0,
    )
    return response.choices[0].message.content

In [10]:
for i, row in data.iterrows():
# for i, row in data[data.index.isin(error_index.index)].iterrows():
    try:
        print(f"Processing row {i}")

        # Process 'abstract' column
        if pd.notna(row['abstract']):
            data.loc[i, "sort_gpt4o_1"] = gpt_extract_abstract(row["doi"], gpt_model, row['abstract'])

        # Process 'abstract_sm' column (ensuring it’s valid and not an error message)
        if pd.notna(row['abstract_sm']) and row['abstract_sm'] != 'Error fetching abstract':
            data.loc[i, "sort_gpt4o_2"] = gpt_extract_abstract(row["doi"], gpt_model, row['abstract_sm'])

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue


Processing row 0
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Processing row 8
Processing row 9
Processing row 10
Processing row 11
Processing row 12
Processing row 13
Processing row 14
Processing row 15
Processing row 16
Processing row 17
Processing row 18
Processing row 19
Processing row 20
Processing row 21
Processing row 22
Processing row 23
Processing row 24
Processing row 25
Processing row 26
Processing row 27
Processing row 28
Processing row 29
Processing row 30
Processing row 31
Processing row 32
Processing row 33
Processing row 34
Processing row 35
Processing row 36
Processing row 37
Processing row 38
Processing row 39
Processing row 40
Processing row 41
Processing row 42
Processing row 43
Processing row 44
Processing row 45
Processing row 46
Processing row 47
Processing row 48
Processing row 49
Processing row 50
Processing row 51
Processing row 52
Processing row 53
Processing row 54
Processing row 55
Pr

## Language Filtering
I see that there're some papers are not written in English. I'll only consider English papers. </br>
Also, I'll add also the columns to distinguish open access papers.

In [13]:
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers

In [ ]:
for id in data[data['oa_link'].isnull()]['id']:
    start = time.time()
    w = Works()[id]
    data.loc[data['id'] == id, 'language'] = w['language']
    if w['language'] != 'en':
        print(f"Language for {id} is {w['language']}")
    data.loc[data['id'] == id, 'is_oa'] = w['open_access']['is_oa']
    data.loc[data['id'] == id, 'oa_status'] = w['open_access']['oa_status']
    data.loc[data['id'] == id, 'oa_link'] = w['open_access']['oa_url']
    if w['open_access']['is_oa'] == True:
        print(f"{id} is open access with {w['open_access']['oa_status']} status")

print(f"✅ Done! Processing {id} took {time.time() - start} seconds")

https://openalex.org/W3202153609 is open access with green status
https://openalex.org/W3107538207 is open access with hybrid status
https://openalex.org/W1510224607 is open access with green status
https://openalex.org/W2474370062 is open access with green status
https://openalex.org/W2476394513 is open access with bronze status
https://openalex.org/W3153342399 is open access with hybrid status
https://openalex.org/W2043203724 is open access with gold status
https://openalex.org/W3096325116 is open access with hybrid status
https://openalex.org/W2560901466 is open access with gold status
https://openalex.org/W2802907016 is open access with hybrid status
https://openalex.org/W2913506912 is open access with bronze status
https://openalex.org/W3158536040 is open access with gold status
https://openalex.org/W3122715779 is open access with green status
https://openalex.org/W4353041084 is open access with hybrid status
https://openalex.org/W2883585018 is open access with bronze status
https

## Preprocess
* Filter out the papers without the abstracts
* Clean the sorting values

perhaps need citation loops to later connect empirical-theoretical papers.
use oa papers + unpaywall api

In [44]:
data_filtered = data[(data['language'] == 'en')]
data_filtered = data[(data['abstract'].isna() == False) | (data['abstract_sm'].isna() == False)]
print(data_filtered.shape)

(7994, 19)


In [13]:
import re

In [21]:
data = pd.read_csv(f"{path}/data_abstracts/filter5_2025-02-26_domain_gpt.csv")

In [30]:
OpenAlex_base = 'sort_gpt_1'
SSCR_base = 'sort_gpt_2'

for i, row in data.iterrows():
    if pd.isna(row[OpenAlex_base]) == False:
        # data_filtered.loc[i, OpenAlex_base] = str(row[OpenAlex_base])
        data.loc[i, OpenAlex_base] = re.sub(r'```(?:plaintext)?|\n', '', row[OpenAlex_base]).strip()
        data.loc[i, OpenAlex_base] = re.sub(r'"(\w+)"', r'\1', row[OpenAlex_base]).lower()
    if pd.isna(row[SSCR_base]) == False:
        # data_filtered.loc[i, SSCR_base] = str(row[SSCR_base])
        data.loc[i, SSCR_base] = re.sub(r'```(?:plaintext)?|\n', '', row[SSCR_base]).strip()
        data.loc[i, SSCR_base] = re.sub(r'"(\w+)"', r'\1', row[SSCR_base]).lower()

In [42]:
print(data[SSCR_base].value_counts())
print(f"\n{data[OpenAlex_base].value_counts()}")
# We'll have to look into the 'error' values and see what's going on.

sort_gpt_2
false          3947
empirical       415
theoretical      87
both             31
Name: count, dtype: int64

sort_gpt_1
false          7070
empirical       663
theoretical     165
both             78
Name: count, dtype: int64


In [41]:
data.loc[data[OpenAlex_base].str.contains('based on|cannot', na=False), OpenAlex_base] = None
data.loc[data[SSCR_base].str.contains('based on|cannot', na=False), SSCR_base] = None

In [28]:
# If you're sure that the values don't contain any error messages (e.g., Based on the....) and if your values still have ```plaintext``` in them, you can remove them by running the following code:
# This searches for the 4 cateogory terms (So don't do this for the error values because they contain "empirical", "theoretical", "both", "false" just to describe the error, not to determine the category.)
valid_terms = {"false", "theoretical", "empirical", "both"}

def extract_valid_term(text):
    match = re.search(r'\b(false|theoretical|empirical|both)\b', text, re.IGNORECASE)
    return match.group(0).lower() if match else ""

for i, row in data.iterrows():
    if pd.notna(row[OpenAlex_base]):
        extracted_value = extract_valid_term(row[OpenAlex_base])
        data.loc[i, OpenAlex_base] = extracted_value

    if pd.notna(row[SSCR_base]):
        extracted_value = extract_valid_term(row[SSCR_base])
        data.loc[i, SSCR_base] = extracted_value


In [3]:
# error_index = data_filtered[(data_filtered['sort_gpt_1'].str.contains('based on the', na=False)) | (data_filtered['sort_gpt_2'].str.contains('based on the', na=False))]
# error_index

📍 They all don't say any opinion. Perhaps re-run GPT.

In [4]:
# data_filtered[(data_filtered['sort_gpt_1'].str.contains('based on the', na=False)) | (data_filtered['sort_gpt_2'].str.contains('based on the', na=False))][['abstract', 'abstract_sm']]

In [ ]:
# data_filtered.to_csv(f"{path}/data_abstracts/final_{today}_gpt4o_filtered.csv", index=False)

### Open Access Value Counts

In [46]:
data_filtered.is_oa.value_counts()

is_oa
False    4769
True     3225
Name: count, dtype: int64

In [47]:
data_filtered[data_filtered['is_oa'] == True].oa_status.value_counts()

oa_status
gold       969
green      849
hybrid     618
bronze     495
diamond    294
Name: count, dtype: int64

In [ ]:
# Check Duplicates #

# data = data.drop_duplicates(subset='id', keep='first')
# print(data)

In [49]:
data_filtered.reset_index(drop=True, inplace=True)
# data

## Filter `false` values

In [51]:
data = pd.read_csv(f"{path}/data_abstracts/filter5_2025-02-26_domain_gpt.csv")

In [52]:
data[data[['sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1', 'sort_gpt4o_2']] == 'false'].count()

Unnamed: 0                    0
id                            0
title                         0
year                          0
doi                           0
landing_page                  0
abstract_inverted_index       0
language                      0
is_oa                         0
oa_status                     0
oa_link                       0
abstract                      0
abstract_sm                   0
authors_sm                    0
domain                        0
sort_gpt_1                 7070
sort_gpt_2                 3947
sort_gpt4o_1               7362
sort_gpt4o_2               4098
dtype: int64

In [53]:
from IPython.display import display

pd.set_option('display.max_colwidth', None)  # Ensures full text display
view_data = data[(data['sort_gpt_1'] == 'false') & (data['sort_gpt_2'] == 'false') & (data['sort_gpt4o_1'] == 'false') & (data['sort_gpt4o_2'] == 'false')][['id', 'title']]
print(len(view_data))
display(view_data.style.set_sticky()) # They look unrelated to intergenerational wealth mobility.

3817


,id,title
1,https://openalex.org/W2145650389,"Inequality, Income Growth, and Mobility: The Basic Facts"
5,https://openalex.org/W3023343113,Creating Networks for Survival and Mobility: Social Capital Among African-American and Latin-American Low-Income Mothers
7,https://openalex.org/W2054649961,Determinants of Mobility in Two-Earner Families: Does the Wife's Income Count?
8,https://openalex.org/W2131625681,Mobility Strategies and Food Shopping for Low-Income Families
10,https://openalex.org/W2109433925,"Trends in income inequality, pro-poor income growth, and income mobility"
12,https://openalex.org/W4242550401,"Low Income, Public Transit, and Mobility"
14,https://openalex.org/W2157648429,"Transit Mobility, Jobs Access and Low-income Labour Participation in US Metropolitan Areas"
15,https://openalex.org/W3044157292,Taxing High-Income Earners: Tax Avoidance and Mobility
20,https://openalex.org/W2157443971,"Low Income, Public Transit, and Mobility"
27,https://openalex.org/W2158507556,Income Mobility and the Earned Income Tax Credit


In [55]:
data_nofalse = data[~data['id'].isin(view_data['id'])]
print(data_nofalse.shape)

(4177, 19)


In [56]:
print(data.shape)

(7994, 19)


In [57]:
view_data2 = data_nofalse[(data_nofalse['sort_gpt_1'] == 'false') & (data['sort_gpt4o_1'] == 'false')][['id', 'title']]
print(len(view_data2))
view_data2 # 2267 rows

3160


/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_86633/3720638019.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  view_data2 = data_nofalse[(data_nofalse['sort_gpt_1'] == 'false') & (data['sort_gpt4o_1'] == 'false')][['id', 'title']]


,id,title
19,https://openalex.org/W3186027345,Mobility patterns are associated with experienced income segregation in large US cities
23,https://openalex.org/W2113431647,Payoffs from Panels in Low-Income Countries: Economic Development and Economic Mobility
25,https://openalex.org/W2051290353,"Earnings Mobility, Family Income and Low Pay"
30,https://openalex.org/W2065069387,Income Taxation and International Mobility
36,https://openalex.org/W2117205490,"Nonfarm diversification, poverty, economic mobility, and income inequality: a case study in village India"
...,...,...
7989,https://openalex.org/W4392119436,Special Report: Lifestyle Psychiatry Emphasizes Behaviors Supporting Mental Health
7990,https://openalex.org/W2326295812,From Disease to Pandemic
7991,https://openalex.org/W4291639851,“Africa's First Woman Press Photographer”: Mabel Cetu's Photographs in<i>Zonk!</i>
7992,https://openalex.org/W4252752336,Comparative Education Review Bibliography 2015


In [ ]:
# data_nofalse.to_csv(f"{path}/data_abstracts/Subset1_four-falses_{today}_filter5.csv", index=False)
# view_data2.to_csv(f"{path}/data_abstracts/Subset2_two-falses_{today}_filter5.csv", index=False)

In [60]:
data_two_falses = data_nofalse[~data_nofalse['id'].isin(view_data2['id'])]
print(data_two_falses.shape)

(1017, 19)


In [ ]:
# data_two_falses.to_csv(f"{path}/data_abstracts/Subset3_no-falses_{today}_filter5.csv", index=False)

# To-Do's

1. Initial number of rows 3713. 
</br>
2. From `final_2025-02-24.csv` data, 231 papers have `false` values in 4 columns (gpt-4o or gpt-4o-mini; based on OpenAlex abstracts or Semantic Scholar + Crossref). ==> This yields 3482 rows (`Subset 1`)  
</br>
3. From (2), 2267 rows have `false` values in 2 columns (gpt-4o or gpt-4o-mini, both based on OpenAlex abstracts) (`Subset 2`) ==> This would yield 1215 rows if we filter them out.  (`Subset 3`)
</br>

✅ Then, first check the 1215 papers:
- Whether their full-texts exist in the journal's API
- and Open-Access tool (Unpaywall).

Depending on the availability, we can also call API on the other 2267 papers or not.
